# Building a Jupyter development environment for microcontrollers


## micropython magic


**-- Taken from https://github.com/v923z/micropython-usermod/blob/master/docs/micropython-usermod.ipynb --**

To make the usage a bit more convenient, we will just register a magic method here to run micropython directly from the notebook. If you don't know what ipython magics are, you can read more at https://ipython.readthedocs.io/en/stable/interactive/magics.html. In any case, we are going to take the contents of a code cell, and pass it to micropython, either on the local machine (unix port), or the bare metal hardware (e.g. the pyboard) as a script.

Note that adding the magic commands makes the python code a wee bit ugly: when running (micro)python with a script file, we won't have so much output as in the interactive console. In fact, except for tracebacks and the results of explicit print statements, we won't see anything at all. For this reason, we will have to call print, whenever we would like to import the results into the notebook. But what the heck! I can definitely put up with that much.


In [10]:
from IPython.core.magic import Magics, magics_class, line_cell_magic
from IPython.core.magic import cell_magic, register_cell_magic, register_line_magic
import subprocess
import os


In [22]:
@register_cell_magic
def micropython(line, cell):
    with open('/snap/bin/micropython.py', 'w') as fout:
        fout.write(cell)
    proc = subprocess.Popen(["./micropython", "/snap/bin/micropython.py"], 
                            stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    print(proc.stdout.read().decode("utf-8"))
    print(proc.stderr.read().decode("utf-8"))
    return None



The following function does nothing outside the notebook: it simply registers a new mode for syntax highlighting, and switches to C, whenever the cell begins with the string %%ccode, or %%makefile.


In [23]:
from IPython.core.display import display_javascript



And finally, at long last, here are the two magic commands. %makefile is simple: each micropython.mk makefile is the same, with the exception of the file name that it is supposed to compile. So, we can take a very generic string, and insert the target. In order to have some trace in the notebook, we also insert the content of the so-generated file into the input field of the cell.

%%ccode reads the contents of the input field of the cell, adds a small header, and writes everything into a file.


In [24]:


@magics_class
class MyMagics(Magics):

    @line_cell_magic
    def makefile(self, line, cell=None):
        raw_cell = "USERMODULES_DIR := $(USERMOD_DIR)\n\n# Add all C files to SRC_USERMOD"
        for _line in cell.split():
            raw_cell += "\nSRC_USERMOD += $(USERMODULES_DIR)/" + _line
            
        raw_cell += "\n\nCFLAGS_USERMOD += -I$(USERMODULES_DIR)"
        with open('../../../usermod/snippets'+line.replace(line.split('/')[-1], 'micropython.mk'), 'w') as mout:
            mout.write(raw_cell)
        self.shell.set_next_input('%%makefile {}\n\n{}'.format(line, raw_cell), replace=True)
        return None
        
    @cell_magic
    def ccode(self, line, cell):
        copyright = """/*
 * This file is part of the micropython-usermod project, 
 *
 * https://github.com/v923z/micropython-usermod
 *
 * The MIT License (MIT)
 *
 * Copyright (c) 2019-2020 Zoltán Vörös
*/
    """
        if line:
            with open('../../../usermod/snippets'+line, 'w') as cout:
                cout.write(copyright)
                cout.write(cell)
            print('written %d bytes to %s'%(len(copyright) + len(cell), line))
            return None

ip = get_ipython()
ip.register_magics(MyMagics)



In [25]:
%%micropython
import os

PermissionError: [Errno 13] Permission denied: '/snap/bin/micropython.py'